In [2]:
import { ChatMessageHistory } from 'langchain/stores/message/in_memory';
import { HumanMessage, AIMessage } from '@langchain/core/messages';
const history = new ChatMessageHistory()
await history.addMessage(new HumanMessage("hi"))
await history.addMessage(new AIMessage("whats can I do for you?"))
const message = await history.getMessages()
message

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "whats can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "whats can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]

In [3]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { Ollama } from "@langchain/community/llms/ollama";
import { ChatMessageHistory } from 'langchain/stores/message/in_memory';
import { HumanMessage, AIMessage } from '@langchain/core/messages';
const chatModel = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3",
});
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`],
    new MessagesPlaceholder("history_message"),
]);

const chain = prompt.pipe(chatModel);
const history = new ChatMessageHistory();
await history.addUserMessage(new HumanMessage("hi, my name is yyl"));
const res1 = await chain.invoke({ history_message: await history.getMessages() });
console.log(res1, 'res1');
await history.addMessage(res1)
await history.addMessage(new HumanMessage("what is my name?"))
const res2 = await chain.invoke({
    history_message: await history.getMessages()
})
console.log(res2, 'res2')

In [ ]:
// 自动维护chat history
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { Ollama } from "@langchain/community/llms/ollama";
import { ChatMessageHistory } from 'langchain/stores/message/in_memory';
import { HumanMessage, AIMessage } from '@langchain/core/messages';
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
const chatModel = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3",
});
const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant. Answer all questions to the best of your ability."],
    new MessagesPlaceholder("history_message"),
    ["human","{input}"]
])
const history = new ChatMessageHistory();
const chain = prompt.pipe(chatModel)
const chainWithHistory = new RunnableWithMessageHistory({
    runnable: chain,
    getMessageHistory: (_sessionId) => history,
    inputMessagesKey: "input",
    historyMessagesKey: "history_message",
})
const res1 = await chainWithHistory.invoke({
    input: "hi, my name is yyl"
},{
    configurable: { sessionId: "none" }
})
console.log(res1)

Hello Yyl! It's nice to meet you. I'm here to help with any questions or topics you'd like to discuss. What brings you here today? Do you have something specific on your mind that you'd like some assistance with?


In [ ]:
const res2 = await chainWithHistory.invoke({
    input: "我的名字叫什么？"
}, {
    configurable: { sessionId: "none" }
})
res2

"You're asking what your name is again! And my answer remains the same: Your name is indeed Yyl!"

In [ ]:
await history.getMessages()

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "hi, my name is yyl",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi, my name is yyl",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hello Yyl! It's nice to meet you. I'm here to help with any questions or topics you'd like to discus"... 112 more characters,
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hello Yyl! It's nice to meet you. I'm here to help with any questions or topics you'd like to discus"... 112 more characters,
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "我的名字叫什么？",
      additional_kwargs: {},
      response_met

In [ ]:
import { RunnableSequence } from "@langchain/core/runnables"
import { StringOutputParser } from "@langchain/core/output_parsers"
const summaryModel = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3",
});
const summaryPrompt = ChatPromptTemplate.fromTemplate(`
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
`);
const summaryChain = RunnableSequence.from([
    summaryPrompt,
    summaryModel,
    new StringOutputParser(),
])
const newSummary = await summaryChain.invoke({
    summary: "",
    new_lines: "I'm 18",
})
await summaryChain.invoke({
    summary: newSummary,
    new_lines: "I'm female"
})

"The conversation has just begun, and the first two pieces of information shared are that the person'"... 40 more characters

In [ ]:
import { RunnableMap, RunnablePassthrough } from "@langchain/core/runnables";
import { getBufferString } from "@langchain/core/messages";
const chatModel = new Ollama({
    baseUrl: "http://localhost:11434", 
    model: "llama3",
});
const chatPrompt = ChatPromptTemplate.fromMessages([
    ["system", `You are a helpful assistant. Answer all questions to the best of your ability.

    Here is the chat history summary:
    {history_summary}
    `],
     ["human","{input}"]
]);
let summary = ""
const history = new ChatMessageHistory();

const chatChain = RunnableSequence.from([
    {
        input: new RunnablePassthrough({
            func: (input) => history.addUserMessage(input)
        })
    },
    RunnablePassthrough.assign({
        history_summary: () => summary
    }),
    chatPrompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (input) => {
            history.addAIChatMessage(input)
            const messages = await history.getMessages()
            const new_lines = getBufferString(messages)
            const newSummary = await summaryChain.invoke({
                summary,
                new_lines
            })
            history.clear()
            summary = newSummary
        }
    })
])
await chatChain.invoke("I'm hungry now")

// const mapChain = RunnableMap.from({
//     a: () => "a",
//     b: () => "b"
// })

// const res = await mapChain.invoke()

// res

In [ ]:
await chatChain.invoke("我今天想吃方便面")

"You're craving instant noodles! That's an easy one to satisfy.\n" +
  "\n" +
  "So, you mentioned wanting to eat ins"... 380 more characters